In [1]:
import zarr # Check version
import numpy as np

zarr.__version__

'2.6.1'

In [2]:
ncar_data = zarr.open("sciserver-experimental/ariel/tests")

In [3]:
ncar_data.info

Name,/
Type,zarr.hierarchy.Group
Read-only,False
Store type,zarr.storage.DirectoryStore
No. members,6
No. arrays,6
No. groups,0
Arrays,"e, p, t, u, v, w"


In [4]:
ncar_data['e'].info

Name,/e
Type,zarr.core.Array
Data type,float32
Shape,"(2048, 2048, 2048)"
Chunk shape,"(256, 256, 256)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,34359738368 (32.0G)
No. bytes stored,13989514907 (13.0G)


<font color="orange">ok, the data is compressed. Let's test access time</font>

In [5]:
chunk_size = (256, 256, 256)
print("Chunk Size: ", np.product(np.array(chunk_size)) / (1024**2), " MB")

Chunk Size:  16.0  MB


## Note if you're using 256^3, you're exactly matching chunk size

In [6]:
a = np.empty(shape=(chunk_size))

## Test access times of 2 different chunks - Energy field

<font color="red">Randal: Why are these 2 runtimes so different? Is this a compression thing?</font>

## Cold Cache - Energy

In [12]:
%%timeit -o -n 1 -r 1

# _ = ncar_data['t'][:256,:256,:256]
_ = ncar_data['e'][256:512,256:512,256:512]

364 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 364 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

<font color="orange">Make sure to only access across chunks if you want to</font>

## Warm Access - Energy

In [21]:
%%timeit -o

# _ = ncar_data['t'][:256,:256,:256]
_ = ncar_data['e'][256:512,256:512,256:512]

71.8 ms ± 1.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


<TimeitResult : 71.8 ms ± 1.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)>

In [22]:
# _ gets previous output
_.all_runs

[0.733253882965073,
 0.7138614639407024,
 0.7114443189930171,
 0.708593490999192,
 0.7022965310607105,
 0.7235285399947315,
 0.7359849340282381]

# Access times - Energy

Warm access time is ~25-30ms

Cold access time is ~100ms

<font color="red">Randal: I Can get it to 600ms+ by accessing different variables - causes disk seek? Which value to pick?</font>

## Trying Medium compressible variable (1.8 Storage Ratio - Temperature

## Cold Cache - Temp

In [34]:
# # Change something makes cold cache?
# ncar_data['t'][256:512,256:512,511] = 0.123

In [3]:
%%timeit -o -n 1 -r 1

a = ncar_data['t'][1024:1280,768:1024,512:768]

728 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 728 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

## Warm access - Temp

In [4]:
%%timeit -o

a = ncar_data['t'][1024:1280,768:1024,512:768]

40.1 ms ± 1.57 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


<TimeitResult : 40.1 ms ± 1.57 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)>

## Cold access - T

In [162]:
%%timeit -o -n 1 -r 1

a = ncar_data['t'][1792:2048,1792:2048,1792:2048]

49.7 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 49.7 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [163]:
%%timeit -o -n 1 -r 1

a = ncar_data['t'][512:768,1792:2048,0:256]

311 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 311 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [164]:
%%timeit -o -n 1 -r 1

a = ncar_data['t'][1792:2048,768:1024,1536:1792]

347 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 347 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [161]:
_.all_runs

[0.04660289699677378]

<font color="red">Smaller variance on this Variable</font>

# Sequential access times - Temperature

Assuming warm access time is ~100ms (and not ~50ms)

Cold access time is ~600ms



## Least compressible var - w (Storage Ratio 1.3)

In [35]:
var = 'w'

In [36]:
ncar_data[var].info

Name,/w
Type,zarr.core.Array
Data type,float32
Shape,"(2048, 2048, 2048)"
Chunk shape,"(256, 256, 256)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,34359738368 (32.0G)
No. bytes stored,27329309819 (25.5G)


In [47]:
%%timeit -o

a = ncar_data[var][:256,:256,:256]

112 ms ± 2.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


<TimeitResult : 112 ms ± 2.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)>

In [48]:
%%timeit -o

a = ncar_data[var][1024:1280,1024:1280,1024:1280]

102 ms ± 1.43 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


<TimeitResult : 102 ms ± 1.43 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)>

In [49]:
%%timeit -o

a = ncar_data[var][1024:1280,768:1024,512:768]

101 ms ± 2.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


<TimeitResult : 101 ms ± 2.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)>

In [50]:
%%timeit -o

a = ncar_data[var][1792:2048,1792:2048,1792:2048]

93.1 ms ± 3.56 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


<TimeitResult : 93.1 ms ± 3.56 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)>

<font color="orange"> Need to create some uncompressed data - skip this on later runs</font>

In [53]:
fsstore = zarr.storage.FSStore('ncar_zarr_uncompressed/e')

In [56]:
ncar_data['e'].shape

(2048, 2048, 2048)

In [59]:
# https://zarr.readthedocs.io/en/stable/api/creation.html#zarr.creation.array

# e_arr = zarr.creation.array(ncar_data['e'], compressor=None)

In [66]:
e_arr.info

Type,zarr.core.Array
Data type,float32
Shape,"(2048, 2048, 2048)"
Chunk shape,"(256, 256, 256)"
Order,C
Read-only,False
Compressor,None
Store type,builtins.dict
No. bytes,34359738368 (32.0G)
No. bytes stored,34359738622 (32.0G)
Storage ratio,1.0


In [69]:
# # https://github.com/zarr-developers/zarr-python/issues/472
# zarr.save_array("ncar_zarr_uncompressed/e", e_arr, compressor=None)

In [70]:
ncar_e_uncompressed = zarr.open("ncar_zarr_uncompressed/e")

In [71]:
ncar_e_uncompressed.info

Type,zarr.core.Array
Data type,float32
Shape,"(2048, 2048, 2048)"
Chunk shape,"(256, 256, 256)"
Order,C
Read-only,False
Compressor,None
Store type,zarr.storage.DirectoryStore
No. bytes,34359738368 (32.0G)
No. bytes stored,34359738622 (32.0G)
Storage ratio,1.0


### Sequential access of Uncompressed

<font color="orange">Timeit runs multiple times. Filesystem should be able to cache better</font>

In [72]:
%%timeit -o

a = ncar_e_uncompressed[:256,:256,:256]

236 ms ± 7.93 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 236 ms ± 7.93 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [73]:
%%timeit -o

a = ncar_e_uncompressed[1024:1280,1024:1280,1024:1280]

209 ms ± 3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 209 ms ± 3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [74]:
%%timeit -o

a = ncar_e_uncompressed[1024:1280,768:1024,512:768]

202 ms ± 2.63 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 202 ms ± 2.63 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [85]:
%%timeit -o

a = ncar_e_uncompressed[1792:2048,1792:2048,1792:2048]

205 ms ± 359 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 205 ms ± 359 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [86]:
_.all_runs

[0.20473615708760917,
 0.20482332108076662,
 0.2051106239669025,
 0.2039962470298633,
 0.2041846140054986,
 0.2043845389271155,
 0.20464796503074467]

### Random access of uncompressed data

<font color="orange">e variable - had 2.5 Compression Ratio</font>

In [76]:
%%timeit -o -n 1 -r 1

a = ncar_e_uncompressed[:256,:256,:256]

209 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 209 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [77]:
%%timeit -o -n 1 -r 1

a = ncar_e_uncompressed[1024:1280,1024:1280,1024:1280]

231 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 231 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [78]:
%%timeit -o -n 1 -r 1

a = ncar_e_uncompressed[1024:1280,768:1024,512:768]

222 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 222 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [79]:
%%timeit -o -n 1 -r 1

a = ncar_e_uncompressed[1792:2048,1792:2048,1792:2048]

222 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 222 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

## t-var. had 1.8 compression ratio. Sequential

In [120]:
# https://github.com/zarr-developers/zarr-python/issues/472
zarr.save_array("ncar_zarr_uncompressed/t", ncar_data['t'], compressor=None)

In [95]:
ncar_e_uncompressed = zarr.open("ncar_zarr_uncompressed/t")
ncar_e_uncompressed.info

Type,zarr.core.Array
Data type,float32
Shape,"(2048, 2048, 2048)"
Chunk shape,"(256, 256, 256)"
Order,C
Read-only,False
Compressor,None
Store type,zarr.storage.DirectoryStore
No. bytes,34359738368 (32.0G)
No. bytes stored,34359738622 (32.0G)
Storage ratio,1.0


In [96]:
%%timeit -o

a = ncar_e_uncompressed[:256,:256,:256]

201 ms ± 4.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 201 ms ± 4.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [97]:
%%timeit -o

a = ncar_e_uncompressed[1024:1280,1024:1280,1024:1280]

207 ms ± 6.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 207 ms ± 6.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [98]:
%%timeit -o -n 1 -r 1

a = ncar_e_uncompressed[1024:1280,768:1024,512:768]

312 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 312 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [99]:
%%timeit -o

a = ncar_e_uncompressed[1792:2048,1792:2048,1792:2048]

200 ms ± 2.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 200 ms ± 2.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

### Random access of uncompressed data

<font color="orange">t variable - had 1.8 Compression Ratio</font>

In [100]:
%%timeit -o -n 1 -r 1

a = ncar_e_uncompressed[:256,:256,:256]

198 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 198 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [101]:
%%timeit -o -n 1 -r 1

a = ncar_e_uncompressed[1024:1280,1024:1280,1024:1280]

202 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 202 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [102]:
%%timeit -o -n 1 -r 1

a = ncar_e_uncompressed[1024:1280,768:1024,512:768]

209 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 209 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [103]:
%%timeit -o -n 1 -r 1

a = ncar_e_uncompressed[1792:2048,1792:2048,1792:2048]

200 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 200 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

## w-var. had 1.3 compression ratio (LOWEST). Sequential

In [110]:
# # https://github.com/zarr-developers/zarr-python/issues/472
# zarr.save_array("ncar_zarr_uncompressed/w", ncar_data['w'], compressor=None)

In [111]:
ncar_e_uncompressed = zarr.open("ncar_zarr_uncompressed/w")
ncar_e_uncompressed.info

Type,zarr.core.Array
Data type,float32
Shape,"(2048, 2048, 2048)"
Chunk shape,"(256, 256, 256)"
Order,C
Read-only,False
Compressor,None
Store type,zarr.storage.DirectoryStore
No. bytes,34359738368 (32.0G)
No. bytes stored,34359738622 (32.0G)
Storage ratio,1.0


In [112]:
%%timeit -o

a = ncar_e_uncompressed[:256,:256,:256]

222 ms ± 9.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 222 ms ± 9.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [113]:
%%timeit -o

a = ncar_e_uncompressed[1024:1280,1024:1280,1024:1280]

203 ms ± 3.72 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 203 ms ± 3.72 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [114]:
%%timeit -o -n 1 -r 1

a = ncar_e_uncompressed[1024:1280,768:1024,512:768]

528 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 528 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [115]:
%%timeit -o

a = ncar_e_uncompressed[1792:2048,1792:2048,1792:2048]

202 ms ± 2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 202 ms ± 2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

### Random access of uncompressed data

<font color="orange">w variable - had 1.3 Compression Ratio (LOWEST)</font>

In [116]:
%%timeit -o -n 1 -r 1

a = ncar_e_uncompressed[:256,:256,:256]

202 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 202 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [117]:
%%timeit -o -n 1 -r 1

a = ncar_e_uncompressed[1024:1280,1024:1280,1024:1280]

202 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 202 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [118]:
%%timeit -o -n 1 -r 1

a = ncar_e_uncompressed[1024:1280,768:1024,512:768]

203 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 203 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [119]:
%%timeit -o -n 1 -r 1

a = ncar_e_uncompressed[1792:2048,1792:2048,1792:2048]

201 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 201 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>